<a href="https://colab.research.google.com/github/Matendy12/Project-4/blob/main/IMDB_Movies_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'imdb-dataset-of-top-1000-movies-and-tv-shows:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1131493%2F1898721%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240417%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240417T095449Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7dd260818c2a780b774d7c9381839b159079016d2e825bb2e9cb0bd7c42c9f5bab456445dfadbdf5896a2b8b4473456dd350f6b05398b6ae194d0594759c633bf40de14ed3e03a98c1c4f5611ef5f10bcd87dff67b5494f63b41e1152178ef6820c9318482f1cad33ec040c081b0c5338dbb07a76a30fa80fecfc106af490f1316fe679220b229993b2e13e8fb965e72fbeed21cb65ad6a0d78912f921981cf71ea45575175bc4bcde0b1fb4829d39acc05e03677e2725d2532d6b8d4a279459b4436606c8f231ed0c7b9b8f728c23be72a8c69d63216db226fd7a5bfe9646a7eaa40416b6fa32bf8f74d3afe127c1ab82e112f0e3c08200f7fefbd12c0d3ae8,data-science-salaries-2023:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3125926%2F5392837%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240417%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240417T095449Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3b7368cbe133495b14200387b7e18fd2e237050cc994682bfbbbc693aae78a5a9471695a91e51fc42fbf559d0a03659e600aeaa4e190ffca96465a89014a66c07132f7d0a11604cee71e34da19ed8ed604bf8b83fe2d35882680b9faa498699c171ca25dc9fcbb7102d314d46e01b4398a965800de5cff857345b3aa95161da00fd21fed01d86904bb17b6dcfd3665350a4646b4536912f5eb961cd1ad4a1b5bde4ac8479066e5bd1846d2c41e9c72fc4f357191d265a8245e57a8c5d7607f6ee7c37d09c2375a31866b7842c77549b85b8e7d6fed00154ed0879016b512276599e4cae5a3091403332b6dbbf185893ec9fce061d1314f84af9d370818715d95'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 179262 bytes downloaded
Downloaded and uncompressed: imdb-dataset-of-top-1000-movies-and-tv-shows
[==================================================] 26034 bytes downloaded
Downloaded and uncompressed: data-science-salaries-2023
Data source import complete.


In [ ]:
import pandas as pd

In [ ]:
movies_df= pd.read_csv("/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv")
movies_df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


In [ ]:
movies_df.columns.tolist()

['Poster_Link',
 'Series_Title',
 'Released_Year',
 'Certificate',
 'Runtime',
 'Genre',
 'IMDB_Rating',
 'Overview',
 'Meta_score',
 'Director',
 'Star1',
 'Star2',
 'Star3',
 'Star4',
 'No_of_Votes',
 'Gross']

In [ ]:
movies_df['Genre'].unique().tolist()

['Drama',
 'Crime, Drama',
 'Action, Crime, Drama',
 'Action, Adventure, Drama',
 'Biography, Drama, History',
 'Action, Adventure, Sci-Fi',
 'Drama, Romance',
 'Western',
 'Action, Sci-Fi',
 'Biography, Crime, Drama',
 'Action, Adventure, Fantasy',
 'Comedy, Drama, Thriller',
 'Adventure, Drama, Sci-Fi',
 'Animation, Adventure, Family',
 'Drama, War',
 'Crime, Drama, Fantasy',
 'Comedy, Drama, Romance',
 'Crime, Drama, Mystery',
 'Crime, Drama, Thriller',
 'Action, Drama, Mystery',
 'Drama, Family, Fantasy',
 'Drama, Music',
 'Biography, Comedy, Drama',
 'Drama, Mystery, Sci-Fi',
 'Biography, Drama, Music',
 'Crime, Mystery, Thriller',
 'Animation, Adventure, Drama',
 'Animation, Drama, War',
 'Adventure, Comedy, Sci-Fi',
 'Horror, Mystery, Thriller',
 'Drama, Romance, War',
 'Comedy, Drama, Family',
 'Animation, Drama, Fantasy',
 'Action, Biography, Drama',
 'Animation, Action, Adventure',
 'Drama, Western',
 'Action, Adventure',
 'Comedy, Drama',
 'Drama, Family',
 'Drama, Mystery, 

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#import tensorflow as tf

In [ ]:
# Drop the non-beneficial ID columns, 'timestamp' and 'NAME'.
movies_df.drop(['No_of_Votes', 'Poster_Link'], axis=1, inplace=True)
movies_df

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,Gross
0,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,"28,341,469"
1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,"134,966,411"
2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"534,858,444"
3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,"57,300,000"
4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,"4,360,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,NaN
996,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,NaN
997,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,"30,500,000"
998,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,NaN


In [ ]:
# Determine the number of unique values in each column.
movies = movies_df.dtypes.index.tolist()
movies_df[movies].nunique()

Series_Title      999
Released_Year     100
Certificate        16
Runtime           140
Genre             202
IMDB_Rating        17
Overview         1000
Meta_score         63
Director          548
Star1             660
Star2             841
Star3             891
Star4             939
Gross             823
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = movies_df[['Series_Title',
                           'Released_Year',
                           'Certificate',
                           'Runtime',
                           'Genre',
                           'IMDB_Rating',
                           'Overview',
                           'Meta_score',
                           'Director',
                           'Star1',
                           'Star2',
                           'Star3',
                           'Star4',
                           'Gross']].value_counts()
app_type_count

Series_Title                                  Released_Year  Certificate  Runtime  Genre                       IMDB_Rating  Overview                                                                                                                                                                                     Meta_score  Director          Star1              Star2                 Star3             Star4                Gross      
(500) Days of Summer                          2009           UA           95 min   Comedy, Drama, Romance      7.7          An offbeat romantic comedy about a woman who doesn't believe true love exists, and the young man who falls for her.                                                                          76.0        Marc Webb         Zooey Deschanel    Joseph Gordon-Levitt  Geoffrey Arend    Chloë Grace Moretz   32,391,374     1
Sicario                                       2015           A            121 min  Action, Crime, Drama        7.6          A

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_count[app_type_count < 600].index)

# Replace in dataframe
for app in application_types_to_replace:
    movies_df['Series_Title'] = movies_df['IMDB_Rating'].replace(app,"Other")

# Check to make sure binning was successful
movies_df['Series_Title'].value_counts()

Series_Title
Other    157
7.8      151
8.0      141
8.1      127
7.6      123
7.9      106
8.2       67
8.3       44
8.4       31
8.5       20
8.6       15
8.7        5
8.8        5
8.9        3
9.0        3
9.2        1
9.3        1
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_count = movies_df['Runtime'].value_counts()
class_count.head()

Runtime
100 min    23
130 min    23
129 min    22
101 min    22
113 min    22
Name: count, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count = movies_df['Series_Title'].value_counts()
class_count_filtered = class_count[class_count > 1]
print(class_count_filtered.head(20))

Series_Title
Other    157
7.8      151
8.0      141
8.1      127
7.6      123
7.9      106
8.2       67
8.3       44
8.4       31
8.5       20
8.6       15
8.7        5
8.8        5
8.9        3
9.0        3
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    movies_df['IMDB_Rating'] = movies_df['Series_Title'].replace(cls,"Other")

# Check to make sure binning was successful
movies_df['IMDB_Rating'].value_counts()

IMDB_Rating
Other    158
7.8      151
8.0      141
8.1      127
7.6      123
7.9      106
8.2       67
8.3       44
8.4       31
8.5       20
8.6       15
8.8        5
8.7        5
9.0        3
8.9        3
9.2        1
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_app_df = pd.get_dummies('Series_Title')
numeric_app_df

,Series_Title
0,True


In [ ]:
# Split our preprocessed data into our features and target arrays
# Replace "Series_Title" with the correct target column name
y = numeric_app_df["Series_Title"]
X = numeric_app_df.drop(columns=["Series_Title"], axis=1)

In [ ]:
print(X.shape)
print(y.shape)

(1, 0)
(1,)


In [ ]:
print(numeric_app_df.shape)

(1, 1)


In [ ]:
import numpy as np
# Increase the dataset size
X = np.concatenate((X, X), axis=0)
y = np.concatenate((y, y), axis=0)

# Split the data again with a smaller test size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=78)

In [ ]:
X = pd.read_csv("/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv")
X.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [ ]:
X_train_df = pd.DataFrame(X_train)
X_train_df.iloc[0].values

array([], dtype=float64)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
import tensorflow as tf

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create a sample dataframe
df = pd.DataFrame({"IMDB_Rating": [1, 2, 3], "Series_Title": [4, 5, 6]})

# Split the dataframe into train and test sets
X_train, X_test = df[["IMDB_Rating", "Series_Title"]].iloc[:1], df[["IMDB_Rating", "Series_Title"]].iloc[1:]

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(1, 2)
(2, 2)


Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 27        
                                                                 
 dense_1 (Dense)             (None, 18)                180       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 226 (904.00 Byte)
Trainable params: 226 (904.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 856ms/step - loss: 0.6931 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6926 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6921 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6916 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6911 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 11ms/step - loss: 0.6907 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6902 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 11ms/step - loss: 0.6897 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6892 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6887 - accuracy: 1.0000


In [ ]:
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"y_test shape: {y_test.shape}")

X_test_scaled shape: (2, 2)
y_test shape: (1,)


In [ ]:
num_samples_x = X_test_scaled.shape[0]
num_samples_y = y_test.shape[0]

if num_samples_x != num_samples_y:
    print("The number of samples in X_test_scaled and y_test are different.")

The number of samples in X_test_scaled and y_test are different.


In [ ]:
# Filter the data
if num_samples_x > num_samples_y:
    X_test_scaled = X_test_scaled[:num_samples_y]
elif num_samples_y > num_samples_x:
    y_test = y_test[:num_samples_x]

# Pad the data
elif num_samples_x < num_samples_y:
    X_test_scaled = np.pad(X_test_scaled, ((0, num_samples_y - num_samples_x), (0, 0)))
elif num_samples_y < num_samples_x:
    y_test = np.pad(y_test, ((0, num_samples_x - num_samples_y), (0, 0)))

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.6641 - accuracy: 1.0000 - 154ms/epoch - 154ms/step
Loss: 0.6641349792480469, Accuracy: 1.0


In [ ]:
# Export our model to HDF5 file
nn.save('Models/IMDBTop1000Movies.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
